In [2]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn

import torchvision
from torchvision.transforms import ToTensor
import torchvision.datasets as datasets

OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_data = datasets.FashionMNIST(root='./',train=True,download=True,transform=ToTensor())
test_data = datasets.FashionMNIST(root='./',train=False,download=True,transform=ToTensor())

CLASSES = train_data.classes

BATCH_SIZE = 32
train_dataloader = DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True)
test_dataloader = DataLoader(test_data,batch_size=BATCH_SIZE,drop_last=True)

IN_FEATURES = 1 #no of color channels
HIDDEN_FEATURES = 32
OUT_FEATURES = len(CLASSES)

class CNNModelV0(nn.Module):
    def __init__(self,in_features: int,hidden_features: int,out_features: int):
        super().__init__()

        #shape =  [32,1,28,28]
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=in_features,
                      out_channels=hidden_features,
                      kernel_size=3,
                      stride=1,
                      padding=1), #shape = [BATCH_SIZE,10,28,28]
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_features,
                      out_channels=hidden_features,
                      kernel_size=3,
                      stride=1,
                      padding=1), #shape = [BATCH_SIZE,10,28,28]
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2), #shape = [BATCH_SIZE,10,14,14]
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_features,
                      out_channels=hidden_features,
                      kernel_size=3,
                      stride=1,
                      padding=1), #shape = [BATCH_SIZE,10,14,14]
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_features,
                      out_channels=hidden_features,
                      kernel_size=3,
                      stride=1,
                      padding=1), #shape = [BATCH_SIZE,10,14,14]
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2), #shape = [BATCH_SIZE,10,7,7]
        )

        self.classifier = nn.Sequential(
            nn.Flatten(), #shape = [BATCH_SIZE*10*7*7], has start_dim=1 by default to preserve the batch dimension accodring to PyTorch convention
            nn.Linear(in_features=hidden_features*7*7,
                      out_features=out_features),
        )
    
    def forward(self,x : torch.Tensor) -> torch.Tensor:
        x = self.conv1(x)
        #print(f"Output shape of Conv1 layer : {x.shape}")
        x = self.conv2(x)
        #print(f"Output shape of Conv2 layer : {x.shape}")
        x = self.classifier(x)
        #print(f"Final output shape of classifer layer : {x.shape}")
        return x


def accuracy_fn(y_true,y_preds):
    x = torch.eq(y_true,y_preds).sum().item()
    return (x/len(y_true)) * 100

def train_model(model : torch.nn.Module,
                dataloader : torch.utils.data.DataLoader,
                loss_fn : torch.nn.Module,
                optimiser : torch.optim,
                device : str,
                accuracy_fn = accuracy_fn,
                ):
   
    model.train()
    epochs = 3
    for epoch in range(epochs):
        train_acc, train_loss = 0,0
        for idx, (x_train,y_train) in enumerate(dataloader):
            x_train, y_train = x_train.to(device), y_train.to(device)
            y_logits = model(x_train) #shape = [32,10]
            y_preds = y_logits.argmax(dim=1).squeeze()
            loss = loss_fn(y_logits,y_train).to(device)
            train_loss += loss
            acc = accuracy_fn(y_preds,y_train)
            train_acc += acc

            optimiser.zero_grad()
            loss.backward()
            optimiser.step()

        train_acc /= len(dataloader)
        train_loss /= len(dataloader)

        print(f"Epoch train loss : {train_loss} | Epoch train acc : {train_acc:.2f}%")

def test_model(model : torch.nn.Module,
               dataloader : torch.utils.data.DataLoader,
               loss_fn : torch.nn.Module,
               optimizer : torch.optim,
               device : str,
               accuracy_fn = accuracy_fn):
    
    model.to(device)
    test_acc, test_loss = 0,0
    for idx, (x_test, y_test) in enumerate(dataloader):
        x_train, y_train = x_train.to(device), y_train.to(device)
        test_logits = model(x_train)
        test_preds = test_logits.argmax(dim=1).squeeze()
        loss = loss_fn(test_logits,y_train)
        test_loss += loss
        acc = accuracy_fn(y_test,test_preds)
        test_acc += acc

    test_acc /= len(dataloader)
    test_loss /= len(dataloader)

    print(f"Test Loss : {test_loss} | Test accuracy : {test_acc:.2f}%")

In [ ]:
model_0 = CNNModelV0(IN_FEATURES,HIDDEN_FEATURES,OUT_FEATURES).to(device)
loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model_0.parameters(),lr=1e-3)

# train_model(model_0,train_dataloader,loss_fn=loss_fn,optimiser=optimiser,device=device)
# test_model(model_0,test_dataloader,loss_fn=loss_fn,optimizer=optimiser,device=device)
model_0

In [ ]:
torch.manual_seed(42)
x = torch.rand(1,28,28)
x = x.unsqueeze(dim=0)
model_1 = CNNModelV0(1,10,10)
print(x.shape)
model_1(x)